<a href="https://colab.research.google.com/github/anishsharma135/Sample_repo/blob/master/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Retrieval using TF-IDF Weighted Rank and TF-IDF Cosine Similarity

## Imports

In [2]:
#Unzipping the stories folder

!unzip stories

Archive:  stories.zip
  inflating: stories/100west.txt     
  inflating: stories/13chil.txt      
  inflating: stories/14.lws          
  inflating: stories/16.lws          
  inflating: stories/17.lws          
  inflating: stories/18.lws          
  inflating: stories/19.lws          
  inflating: stories/20.lws          
  inflating: stories/3gables.txt     
  inflating: stories/3lpigs.txt      
  inflating: stories/3sonnets.vrs    
  inflating: stories/3student.txt    
  inflating: stories/3wishes.txt     
  inflating: stories/4moons.txt      
  inflating: stories/5orange.txt     
  inflating: stories/6ablemen.txt    
  inflating: stories/6napolen.txt    
  inflating: stories/7oldsamr.txt    
  inflating: stories/7voysinb.txt    
   creating: stories/FARNON/
  inflating: stories/FARNON/.windex.html  
 extracting: stories/FARNON/.descs   
  inflating: stories/FARNON/.footer  
  inflating: stories/FARNON/.header  
  inflating: stories/FARNON/index.html  
   creating: stories/SRE/
  i

In [3]:
#Installing num2words library - It converts 42 to forty-two
pip install num2words


     |████████████████████████████████| 102kB 2.9MB/s 


In [0]:
#Importing required libraries 

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

# %load_ext autotime

In [0]:
#declaring values of variables title and alpha

title = "stories"
alpha = 0.3

## Taking all folders

In [0]:
#code to traverse through all the folder directories

folders = [x[0] for x in os.walk(str(os.getcwd())+'/'+title+'/')]
folders[0] = folders[0][:len(folders[0])-1]

In [7]:
folders

['/content/stories', '/content/stories/SRE', '/content/stories/FARNON']

## Collecting the file names and titles

In [8]:
#By traversing through the folders we are locating all the text files and adding all the text files into dataset

dataset = []

c = False

for i in folders:
    file = open(i+"/index.html", 'r')
    text = file.read().strip()
    file.close()

    file_name = re.findall('><A HREF="(.*)">', text)
    file_title = re.findall('<BR><TD> (.*)\n', text)

    if c == False:
        file_name = file_name[2:]
        c = True
        
    print(len(file_name), len(file_title))

    for j in range(len(file_name)):
        dataset.append((str(i) +"/"+ str(file_name[j]), file_title[j]))

452 452
15 15
0 0


In [9]:
len(dataset) #printing length of dataset


467

In [0]:
N = len (dataset) #initializing variable n with value of length of Dataset

In [0]:
def print_doc(id):
    print(dataset[id])
    file = open(dataset[id][0], 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    

In [12]:
print(text)

<HTML>
<TITLE>T E X T F I L E S</TITLE>
<BODY BGCOLOR="#000000" TEXT="#00FF00" LINK="#00FF00" ALINK="#00AA00" VLINK="#00AA00">
<H1>Stories: TRISTAN FARNON</H1>
<P>
<TABLE WIDTH=100%>
<TD BGCOLOR=#00FF00><FONT COLOR=#000000><B>Filename</B><BR></FONT></TD>
<TD BGCOLOR=#00DD00><FONT COLOR=#000000><B>Size</B><BR></FONT></TD>
<TD BGCOLOR=#00AA00><FONT COLOR=#000000><B>Description of the Textfile</B><BR></TD></TR>

<tab indent=60 id=T><br>
</TABLE><P><TABLE WIDTH=100%></TABLE><P>
At the request of Tristan Farnon, this directory has been removed.
<P>
This saddens me no end, because his stories were truly wonderful. He'd
post them on the Dark Side of the Moon AE, and while there were always
interesting files being posted there, Tristan's always caused a rush of
excitement through the place. You could tell he'd posted one, because 
the system would be really busy. 
<P>
Tristan's stories were filled with irony, interesting situations, and
well-written dialogue, all pleasant changes from what els

# Preprocessing

In [0]:
#creating function to convert data to lower case
def convert_lower_case(data):
    return np.char.lower(data)

In [0]:
#creating function to remove stop words from data
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [0]:
#functions to remove punctuation

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [0]:
#function to remove apostrophe
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [0]:
#function to reduce words from long to stem words - like removing "-ing" from creating to create word "create"

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [0]:
#In this function we are using the num2words library

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [0]:
#We created a function preprocess and passed data as argument. In this function we called all the preprocessing functions we defined above

def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

## Extracting Data

In [20]:
#loading nltk library
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
processed_text = [] #creating empty array
processed_title = [] #creating empty array

for i in dataset[:N]: #traversing through the dataset
    file = open(i[0], 'r', encoding="utf8", errors='ignore')
    text = file.read().strip()
    file.close()

    processed_text.append(word_tokenize(str(preprocess(text)))) #preprocessing the text by using the preprocessing function and appending it to processed_text
    processed_title.append(word_tokenize(str(preprocess(i[1])))) #preprocessing the title by using the preprocessing function and appending it to processed_text

## Calculating DF for all words

In [0]:

DF = {} #Document Frequency variable 

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i) #Adding tokens from text from dataset to DF array
        except:
            DF[w] = {i}

    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i) #Adding tokens from title from dataset to DF array
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [24]:
 DF

{'sharewar': 5,
 'trial': 35,
 'project': 63,
 'freewar': 1,
 'need': 243,
 'support': 87,
 'continu': 193,
 'one': 444,
 'hundr': 329,
 'west': 65,
 'fifti': 160,
 'three': 293,
 'north': 73,
 'jim': 20,
 'prentic': 3,
 'copyright': 115,
 'thousand': 314,
 'nine': 284,
 'nineti': 215,
 'brandon': 3,
 'manitoba': 2,
 'canada': 10,
 'magic': 89,
 'phrase': 40,
 'spoken': 42,
 'mumbl': 45,
 'thought': 323,
 'inwardli': 8,
 'soul': 104,
 'ventur': 57,
 'northward': 7,
 'imaginari': 20,
 'line': 186,
 'shown': 64,
 'map': 48,
 'label': 22,
 'degr': 56,
 'presenc': 83,
 'indic': 74,
 'highway': 35,
 'travel': 109,
 'road': 119,
 'side': 243,
 'sign': 151,
 'divi': 29,
 'territori': 21,
 'distinct': 41,
 'mind': 234,
 'intern': 50,
 'border': 37,
 'writer': 55,
 'poet': 17,
 'pilot': 36,
 'contribut': 31,
 'lore': 6,
 'rigor': 6,
 'life': 274,
 'bush': 45,
 'told': 246,
 'tale': 73,
 'man': 271,
 'eat': 139,
 'mosquito': 6,
 'murder': 81,
 'hord': 15,
 'black': 189,
 'fli': 121,
 'lump': 25,

In [0]:
total_vocab_size = len(DF) #assigning length of data frequency array to total vocab size

In [26]:
total_vocab_size

32350

In [0]:
total_vocab = [x for x in DF] #assigning token one by one to the total_vocab variable

In [28]:
print(total_vocab[:20])

['sharewar', 'trial', 'project', 'freewar', 'need', 'support', 'continu', 'one', 'hundr', 'west', 'fifti', 'three', 'north', 'jim', 'prentic', 'copyright', 'thousand', 'nine', 'nineti', 'brandon']


In [0]:
# defining doc_freq function and passing the words as argument
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

### Calculating TF-IDF for body, we will consider this as the actual tf-idf as we will add the title weight to this.

In [0]:
doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count #calculating the value of tf
        df = doc_freq(token) #Calculating the value of df using function doc_freq
        idf = np.log((N+1)/(df+1)) #calculating the idf using the log function
        
        tf_idf[doc, token] = tf*idf

    doc += 1

In [0]:
# tf_idf

### Calculating TF-IDF for Title

In [0]:
#Performing same operations as above but this time for the title dataset
doc = 0

tf_idf_title = {}

for i in range(N):
    
    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1)) #numerator is added 1 to avoid negative values
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1

In [0]:
# tf_idf_title

In [43]:
tf_idf[(0,"go")] #Checking the value of the token go and calculating its idf value

0.0002906893990853149

In [44]:
tf_idf_title[(0,"go")] #Checking the value of the token "go" in title dataset and calculating its idf value

0.0002906893990853149

## Merging the TF-IDF according to weights

In [0]:
#using weights with the help of variable alpha which we set manual value in the beginning
for i in tf_idf:
    tf_idf[i] *= alpha

In [0]:
for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]

In [47]:
len(tf_idf)

344378

# TF-IDF Matching Score Ranking

In [48]:
#Maching the score using the weights 

def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}

    for key in tf_idf: #traversing through tf_idf
        # If the word is in the dictionary 
        #adding word to tf_idf list
        if key[1] in tokens: 
            try:
                query_weights[key[0]] += tf_idf[key]
                #if the word is found at the first time
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    l = []
    
    for i in query_weights[:10]:
        l.append(i[0])
    
    print(l)
    
#Depending on the value of the tf_idf the top 10 are ranked
matching_score(10, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")

Matching Score

Query: Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying

['without', 'drive', 'rebeccah', 'insist', 'kate', 'lost', 'momentum', 'stood', 'next', 'slat', 'oak', 'bench', 'canist', 'still', 'clutch', 'survey']

[166, 200, 352, 433, 211, 350, 175, 187, 188, 294]


In [49]:
print_doc(166)

('/content/stories/fea3', 'Survey Results, by Joe DeRouen (1994)')


# TF-IDF Cosine Similarity Ranking

In [0]:
#define cosin similarity
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

### Vectorising tf-idf

In [0]:
#Calculating dimension 
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [0]:
#Generates number representation for token

def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [53]:
#calculating cosin similarity and finding the matching documents
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    print(out)

#     for i in out:
#         print(i, dataset[i][0])

Q = cosine_similarity(10, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")

Cosine Similarity

Query: Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying

['without', 'drive', 'rebeccah', 'insist', 'kate', 'lost', 'momentum', 'stood', 'next', 'slat', 'oak', 'bench', 'canist', 'still', 'clutch', 'survey']

[200 166 433 175 169 402 211  87 151 369]


In [0]:
print_doc(200)

('/content/stories/ghost', 'Time for Flowers, by Gay Bost')
TIME FOR FLOWERS
  by Gay Bost

They'd put flowers up. She hadn't noticed. Time wouldn't hold still.
She remembered, quite clearly, that time had been a simple thing; one
moment following the previous one, seconds strung out neatly like her
mother's pearls laid out on the dark mahogany vanity each Sunday
morning. But there had been a catch . . . 

Hung around Mother's neck the catch clicked and the tidy little line 
of seconds became a never ending circle with only the catch in the 
middle. For some reason the thought of pearls gathered from the sea, 
naturally nested within the confines of oyster shells, scattered 
haphazardly about the ocean floor disturbed her.

Now they'd put up the flowers in the same careless groupings. This,
too, disturbed her. Bright yellow trumpets, their collars spread to
catch the sun, dotted the front yard in clusters of two or three, five
or six. Bunches laid carelessly and forgotten. In a moment 